<a href="https://colab.research.google.com/github/changsin/ClassifyImages/blob/main/notebooks/dedupe_images.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Scalable solutions to detect duplicate images

Identical images are easier to detect if identity means pixel-wise idenity. We can use image hash, for instance, to encode each image and quickly compare hash values of two images. However, if we want to compare how similar two images are or whether two images "very similar" to the point where human beings cannot tell the difference, then the problem is much harder. The latter is the working definition of "duplication" and I want to show how you can detect duplicate images.



# Problems
The first question to ask is why you want to detect duplicate images. There could be many reasons, but let me list a few reasons for machine learning purposes.

**1. Bias:** Having duplicate data means that the model will be trained more on that type of data and thus will be biased. One or more duplicate data might be Okay but if there are a lot of them, the model will fail to generalize on new data.

**2. Cost:** Each piece of data needs to be labeled manually and reviewed one way or the other. Having duplicate data means adding an unnecessary cost for data processing, storage, and computation.

**3. Noise:** Duplicate data can lead to subtle noise in the training set too if the labeling is not done consistently or correctly.

For theses reasons, we want to remove as much duplicate data as possible. However, detecting and removing duplicates manually is not scalable and thus the need for a better approach.

# Other Approaches
There are a few known solutions to detect duplicates.

## 1. Image hash
Encoding each image as a hash value and then comparing hash values is a quick and easy method to check identical images. However, as you will see, this method does not work if the image is slightly altered (e.g., the image pixels are moved one pixel left). Besides, image hash only provides a binary answer: the same or not. What we need is a similarity measure.

## 2. Traditional approach
Then there are traditional approaches to detect images similarities like taking differences of pixel values, comparing histograms, calculating structural similarity index [SSIM](https://en.wikipedia.org/wiki/Structural_similarity), or [feature matching](https://medium.com/data-breach/introduction-to-feature-detection-and-matching-65e27179885d). The pros and cons of each approach is beyond the scope of the current article, but in general they are good for comparing two or a few images at a time but hard to use at a large scale.
 
## 3. Deep Learning
The third approach is to leverage Deep Learning to find duplicates: e.g., a [Siamese network](https://conferences.oreilly.com/strata/strata-eu-2018/cdn.oreillystatic.com/en/assets/1/event/267/Using%20Siamese%20CNNs%20for%20removing%20duplicate%20entries%20from%20real%20estate%20listing%20databases%20Presentation.pdf). While this approach is the most robust and can handle even rotated duplicate images, it requires the heavy lifting of training and inferencing using another neural network.


# Requirements
In light of the survey of other approaches, here are the requirements that we would like to have:

1. Similarity meausres: Instead of a single binary decision of match or non-match, we want metrics that can tell us how similar two given images are.
2. Scalable: The solution should work on two images, multiple images, and a large dataset of images.
3. Light-weight: Removing duplicates is for pre-processing training data for Deep Learning so we want the solution to be as light-weight as possible.

# Solution

The solution I propose is a modified clustering algorithm. The steps are:

1. Cluster an initial dataset based on feature maps
2. Save the centroid values.
3. Load the centroids when processing additional image data
4. For each image, compare its feature map with centroids and find the cluster it belongs to.
5. Compare similarity measures with other images within the cluster.

To make it scalable, each cluster should be kept at a few hundred images.

# Implementation

For the actual implementation, first we will pass images through a CNN (Convolutional Neural Network) to extract the feature maps and then use them to find clusters.

## Image hash demo

The image hash works well to find identical images.

In [ ]:
!git clone https://github.com/changsin/ClassifyImages.git

Cloning into 'ClassifyImages'...
remote: Enumerating objects: 1078, done.
remote: Counting objects: 100% (1078/1078), done.
remote: Compressing objects: 100% (942/942), done.
remote: Total 1078 (delta 158), reused 911 (delta 46), pack-reused 0
Receiving objects: 100% (1078/1078), 241.64 MiB | 16.84 MiB/s, done.
Resolving deltas: 100% (158/158), done.


In [ ]:
import hashlib

# https://stackoverflow.com/questions/26000198/what-does-colon-equal-in-python-mean
def get_hash(img_path):
  # This function will return the `md5` checksum for any input image.
  with open(img_path, "rb") as f:
    img_hash = hashlib.md5()
    chunk = f.read()
    while chunk:
      img_hash.update(chunk)
      chunk = f.read()
  return img_hash.hexdigest()

sample_image_path = '/content/ClassifyImages/data/test/david-brooke-martin-t_ZdxJsE8iM-unsplash.jpg'
get_hash(sample_image_path)

'aeba119dcd4359b1d24c187fce013941'

In [ ]:
import cv2

x = cv2.imread(sample_image_path)

x.shape

(800, 1200, 3)

However, if you slightly modify the image by removing a single column of pixels, the two images look the same but the image has is completely different.

In [ ]:
cv2.imwrite('test.jpg', x[1:, 1:])

True

In [ ]:
get_hash('test.jpg')

'0d2bbd567c141e45e33b29da07d3851e'

In [ ]:
import matplotlib.pyplot as plt

plt.imshow(x)

In [ ]:
plt.imshow(x[1:, 1:])

## Cluster Solution
To handle duplicate images, we need a more robust solution. As outlined above, the first step is to create clusters out of the input images.

For a demo purpose, I am using a short video clip of a river scenery. On this particular day, I was lucky enough to witness a double rainbow hung along the Han River in Seoul just when the sun was going down. The video clip will show both the double rainbow and the sunset in a single take. What I expect is that there should be two clusters. The first is about the double rainbow and the second the sunset. Let's see if the clustering algorithm can do that. 
Now let's see how 

### Sample images

Let's download the demo clip and extract frame images.

In [ ]:
!pip install -q youtube-dl

In [ ]:
from IPython.display import YouTubeVideo

rainbow_sunset = "rainbow_sunset"
rainbow_sunset_id = 'I1wDZICq8XY'
YouTubeVideo(rainbow_sunset_id)

In [ ]:
def download_youtube(youtube_id, save_filename):
  !youtube-dl -f 'bestvideo[ext=mp4]' --output $save_filename".%(ext)s" https://www.youtube.com/watch?v=$youtube_id

download_youtube(rainbow_sunset_id, rainbow_sunset)

[youtube] I1wDZICq8XY: Downloading webpage
[youtube] I1wDZICq8XY: Downloading MPD manifest
[download] Destination: rainbow_sunset.mp4
[download] 100% of 1.26MiB in 00:28


In [ ]:
def to_images(youtube_id, save_folder):
  !test -d $save_folder && rm $save_folder/*
  !mkdir $save_folder
  !ffmpeg -i $youtube_id".mp4" -filter:v fps=10 $save_folder/out%05d.jpg

to_images(rainbow_sunset, rainbow_sunset)

ffmpeg version 3.4.8-0ubuntu0.2 Copyright (c) 2000-2020 the FFmpeg developers
  built with gcc 7 (Ubuntu 7.5.0-3ubuntu1~18.04)
  configuration: --prefix=/usr --extra-version=0ubuntu0.2 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --enable-gpl --disable-stripping --enable-avresample --enable-avisynth --enable-gnutls --enable-ladspa --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librubberband --enable-librsvg --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libssh --enable-libtheora --enable-libtwolame --enable-libvorbis --enable-libvpx --enable-libwavpack --enable-libwebp --enable-libx265 --enable-libxml2 --enable-libxvid --enable-lib

### Extract feature maps

In [ ]:
import glob
import os

import cv2
import numpy as np
import matplotlib.pyplot as plt

from keras.applications.vgg16 import VGG16
from keras.applications.vgg16 import preprocess_input
from keras.models import Model
from scipy.spatial.distance import cdist
from sklearn import preprocessing  # to normalise existing X
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA

In [ ]:
"""
Methods for loading and visualizing images
"""

IMAGE_SIZE = 320

def glob_files(folder, file_type='*'):
    search_string = os.path.join(folder, file_type)
    files = glob.glob(search_string)

    # print('searching ', path)
    paths = []
    for f in files:
      if os.path.isdir(f):
        sub_paths = glob_files(f + '/')
        paths += sub_paths
      else:
        paths.append(f)

    # We sort the images in alphabetical order to match them
    #  to the annotation files
    paths.sort()

    return paths

def load_images(path, file_type="*"):
    files = glob_files(path, file_type)

    images = []
    for file in files:
        # print(file)
        image = cv2.imread(file)
        if image is not None:
            image = cv2.resize(image, (IMAGE_SIZE, IMAGE_SIZE))
            # normalize
            image = image / 256
            images.append(image)
        else:
            print(file, ' is not an image file')

    return np.array(images)

def plot_images(X, idx=None, limit=20):
  fig = plt.figure(figsize=(50,60))

  # The number of images for plotting is limited to 50
  end_id = len(X) if len(X) < limit else limit
  if idx is None:
    idx = range(0, end_id)

  i = 0
  for id in idx:
    axis = fig.add_subplot(5, 5, i+1)
    plt.axis('off')
    image = X[id]
    plt.imshow(image)
    i += 1

In [ ]:
images = load_images(rainbow_sunset)
images.shape
plot_images(images, limit=10)

# Cluster

In [ ]:
def get_pca_reduced(X_features, dimensions=2):
  X_features_flatten = X_features.reshape(X_features.shape[0], -1)
  pca = PCA(dimensions)

  X_features_pca_reduced = pca.fit_transform(X_features_flatten)

  return X_features_pca_reduced, pca


def get_clusters(X_reduced, K):
  kmeans = KMeans(n_clusters=K, random_state=0)
  X_clusters = kmeans.fit(X_reduced)

  return X_clusters, kmeans

def to_cluster_idx(cluster_labels, bins):
    """
    param labels: cluster labels
    param bins: range of K
    returns: dictionary of cluster IDs
    """
    cluster_dict = dict()
    for cluster_id in bins:
        cluster_dict[cluster_id] = np.where(cluster_labels == cluster_id)[0]
    return cluster_dict

def cluster_images(path, K=2):
  X = load_images(path)
  plot_images(X)
  X_reduced, pca = get_pca_reduced(X, dimensions=K)

  X_clusters, kmeans = get_clusters(X_reduced, K)

  # get the image ids of each cluster
  cluster_idx = to_cluster_idx(X_clusters.labels_, range(K))

  # keep the cluster centers
  print(kmeans.cluster_centers_)
  print(cluster_idx)
  
  return X_reduced, kmeans

In [ ]:
import matplotlib.pyplot as plt

def plot_data_in_clusters(data, kmeans, idx=None, show_centroids=True):
  marker_size = 7

  # Plot the decision boundary. For that, we will assign a color to each
  x_min, x_max = data[:, 0].min(), data[:, 0].max()
  y_min, y_max = data[:, 1].min(), data[:, 1].max()

  # Step size of the mesh. Decrease to increase the quality of the VQ.
  # point in the mesh [x_min, x_max]x[y_min, y_max].
  h = float((x_max - x_min)/100)

  PADDING = h * marker_size
  x_min, x_max = x_min - PADDING, x_max + PADDING
  y_min, y_max = y_min - PADDING, y_max + PADDING

  xx, yy = np.meshgrid(np.arange(x_min, x_max, h), np.arange(y_min, y_max, h))

  # Obtain labels for each point in mesh. Use last trained model.
  Z = kmeans.predict(np.c_[xx.ravel(), yy.ravel()])

  # Put the result into a color plot
  Z = Z.reshape(xx.shape)

  plt.figure(2)
  # plt.clf()
  plt.imshow(Z, interpolation="nearest",
              extent=(xx.min(), xx.max(), yy.min(), yy.max()),
              cmap=plt.cm.Paired, aspect="auto", origin="lower")

  plt.plot(data[:, 0], data[:, 1], 'k.', markersize=marker_size)

  if show_centroids:
    markers = ["o", "1"]
    # Plot the centroids as a white X
    centroids = kmeans.cluster_centers_

    for id in range(len(centroids)):
      c = centroids[id]
      plt.scatter(c[0], c[1], marker=markers[id], s=150, linewidths=marker_size,
                  color="w", zorder=10)
  if idx:
    for id in idx:
        plt.scatter(data[id, 0], data[id, 1], marker="x",
                    s=150, linewidths=marker_size,
                    color="w", zorder=10)

  plt.title("K-means clustering")
  plt.xlim(x_min, x_max)
  plt.ylim(y_min, y_max)
  plt.xticks(())
  plt.yticks(())
  plt.show()

In [ ]:
X_reduced_rs, kmeans_rs = cluster_images(rainbow_sunset)
plot_data_in_clusters(X_reduced_rs, kmeans=kmeans_rs, idx=[1])

In [ ]:
plot_images(images, idx=[55, 56, 57])

## Save the centroids

# A new video - migrating birds

In [ ]:
migrating_birds = "migrating_birds"
migrating_birds_id = '-0jhgfyzINQ'
YouTubeVideo(migrating_birds_id)

In [ ]:
download_youtube(migrating_birds_id, migrating_birds)
to_images(migrating_birds, migrating_birds)

[youtube] -0jhgfyzINQ: Downloading webpage
[youtube] -0jhgfyzINQ: Downloading MPD manifest
[download] Destination: migrating_birds.mp4
[download] 100% of 2.36MiB in 00:00
rm: cannot remove 'migrating_birds/*': No such file or directory
mkdir: cannot create directory ‘migrating_birds’: File exists
ffmpeg version 3.4.8-0ubuntu0.2 Copyright (c) 2000-2020 the FFmpeg developers
  built with gcc 7 (Ubuntu 7.5.0-3ubuntu1~18.04)
  configuration: --prefix=/usr --extra-version=0ubuntu0.2 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --enable-gpl --disable-stripping --enable-avresample --enable-avisynth --enable-gnutls --enable-ladspa --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --en

In [ ]:
def to_json(path, data):
    """
    save json data to path
    """
    with open(path, 'w', encoding='utf-8') as file:
        json.dump(data, file, ensure_ascii=False, indent=4)

def from_json(path):
    """
    save json data to path
    """
    file = open(path, 'r', encoding='utf-8')
    return json.load(file)

In [ ]:
images_mb = load_images(migrating_birds)
images_mb.shape
plot_images(images_mb, limit=10)

In [ ]:
kmeans_rs.cluster_centers_

array([[ 79.34138652,  -4.37693964],
       [-41.32363881,   2.27965606]])

In [ ]:


# Calculate distances of all points
distances = cdist(X_train_pca, X_train_pca)